ランタイムをGPUモードに変更

In [ ]:
# !pip install face_recognition

ライブラリーのインポート

In [ ]:
# -*- coding: utf-8 -*-
import os
import sys
import face_recognition
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import glob

設定ファイルの読み込み

In [ ]:
import config
emp_info = config.emp_info
threshold = config.threshold
mode = config.mode

顔情報の初期化

In [ ]:
face_locations = []
face_encodings = []

登録画像の読み込み

In [ ]:
# image_jpフォルダーに認証したい顔画像及び認証時に表示させたい名前に変更する。
# ※フォルダーに入ってる画像は架空の人物及び名前を使用しています。


# image_paths = glob.glob('image/*')
image_paths = glob.glob('image_jp/*')
image_paths.sort()
known_face_encodings = []
known_face_names = []
checked_face = []

In [ ]:
for image_path in image_paths:
    im_name = os.path.basename(image_path).split('.')[0]
    image = face_recognition.load_image_file(image_path)
    face_encoding = face_recognition.face_encodings(image)[0]
    known_face_encodings.append(face_encoding)
    known_face_names.append(im_name)

In [ ]:
video_capture = cv2.VideoCapture(0)

In [ ]:
def main(): 

	while True:
		# ビデオの単一フレームを取得
		_, frame = video_capture.read()

		# ビデオの現在のフレーム内のすべての顔に対してその位置情報を検索
		face_locations = face_recognition.face_locations(frame)
		# 顔の位置情報からエンコードを生成
		face_encodings = face_recognition.face_encodings(frame, face_locations)

		for face_encoding in face_encodings:
			# 顔が登録済みの顔と一致するか確認
			matches = face_recognition.compare_faces(known_face_encodings, face_encoding, threshold)
			name = "Unknown"

			# カメラ画像と最も近い登録画像を見つける
			face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
			best_match_index = np.argmin(face_distances)
			if matches[best_match_index]:
				name = known_face_names[best_match_index]

		# 位置情報の表示
		for (top, right, bottom, left) in face_locations:
        	# 顔領域に枠を描画
			cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

			# 枠の下に名前を表示
			cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
			# font = cv2.FONT_HERSHEY_DUPLEX
			# cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
   
			# 日本語表示部分
			fontpath ='meiryo.ttc'
			font = ImageFont.truetype(fontpath, 32) # フォントサイズ32pt
			img_pil = Image.fromarray(frame) # 配列の各値を8bit(1byte)整数型(0～255)をPIL Imageに変換。
			draw = ImageDraw.Draw(img_pil) # drawインスタンスを生成
			position = (left + 6, bottom - 40) # テキスト表示位置
			draw.text(position, name, font = font , fill = (255, 255, 255, 0) ) # drawにテキストを記載 fill:色 B,G,R,Alpha設定は反映されない
			frame = np.array(img_pil) # PIL を配列に変換

			# 本人確認
			if mode == 1 and name != "Unknown":
				check_password(name)

		# 結果をビデオに表示
		cv2.imshow('Video', frame)

		# ESCキーで終了
		if cv2.waitKey(1) == 27:
			break

In [ ]:
def check_password(name):
    if name in checked_face:
        return

    emp_id = input (name + "さんの社員番号を入力してください。")
    
    if emp_info[name] == emp_id:
        print("出勤しました")
        checked_face.append(name)
    else:
        print("社員番号が不正です")
        name = ""

In [ ]:
main()

In [ ]:
# ウェブカメラへの操作を開放
video_capture.release()
cv2.destroyAllWindows()